Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.79-cp310-cp310-linux_x86_64.whl size=3663687 sha256=5587693214008f89cddde90677b7140e78b51bf169ab26b763700140f97e39f6
  Stored in directory: /root/.cache/pip/wheels/bb/2e/11/8b10c6b698e6abc1289e9919e098ac4bcf6b16ebd46153e8ba
Successfully built llama-cpp-python


In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
llm = Llama(
    model_path="/kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf",
    n_ctx=4096,      # Max tokens for in + out
    n_threads=4,     # CPU cores used
    n_gpu_layers=-1,  # Load all layers into VRAM of the GPU
)
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 lla

Model load time: 56.6172 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.chat_template': "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}", 'tokenizer.ggml.eos_token_id': '128009', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'general.architecture': 'llama', 'llama.rope.freq_base': '500000.000000', 'llama.context_length': '8192', 'general.name': '.', 'llama.vocab_size': '128256', 'general.file_type': '17', 'llama.embedding_length': '4096', 'lla

In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm(prompt="Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    4075.66 ms
llama_print_timings:      sample time =      44.06 ms /    20 runs   (    2.20 ms per token,   453.87 tokens per second)
llama_print_timings: prompt eval time =    4075.49 ms /    15 tokens (  271.70 ms per token,     3.68 tokens per second)
llama_print_timings:        eval time =    7026.59 ms /    19 runs   (  369.82 ms per token,     2.70 tokens per second)
llama_print_timings:       total time =   11177.36 ms /    34 tokens


What are the names of three European capital cities?
['1. Paris, France; 2. Rome, Italy; 3. London, UK.']
time: 11.1850


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=1000)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4075.66 ms
llama_print_timings:      sample time =       4.71 ms /     2 runs   (    2.36 ms per token,   424.36 tokens per second)
llama_print_timings: prompt eval time =    5947.90 ms /    24 tokens (  247.83 ms per token,     4.04 tokens per second)
llama_print_timings:        eval time =     363.51 ms /     1 runs   (  363.51 ms per token,     2.75 tokens per second)
llama_print_timings:       total time =    6320.05 ms /    25 tokens


What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
['1.']
time: 6.3283


In [5]:
r0, t0 = ask(query='How do I drive from Washington DC to New York City?', max_tokens=3000)
print(r0)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4075.66 ms
llama_print_timings:      sample time =     327.48 ms /   149 runs   (    2.20 ms per token,   454.99 tokens per second)
llama_print_timings: prompt eval time =    3787.99 ms /    15 tokens (  252.53 ms per token,     3.96 tokens per second)
llama_print_timings:        eval time =   56773.63 ms /   148 runs   (  383.61 ms per token,     2.61 tokens per second)
llama_print_timings:       total time =   61114.91 ms /   163 tokens


['1. From Washington DC, take I-495 N towards Baltimore. 2. Continue on I-495 N for approximately 30 miles until you reach the Capital Beltway. 3. Take the exit for I-95 N/US-1 N towards Baltimore/New York City. 4. Stay on I-95 N for about 140 miles until you reach the George Washington Bridge in New York City. 5. Cross the bridge and take the exit for I-78 W/US-1 S towards Newark. 6. Continue on I-78 W/US-1 S for approximately 10 miles until you reach downtown Newark. From there, you can access various parts of the city using local roads and highways.']


In [6]:
r1, t1 = ask(query='What would I need to do to build an edible bicycle?', max_tokens=3000)
print(r1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4075.66 ms
llama_print_timings:      sample time =     180.66 ms /    77 runs   (    2.35 ms per token,   426.22 tokens per second)
llama_print_timings: prompt eval time =    3734.29 ms /    15 tokens (  248.95 ms per token,     4.02 tokens per second)
llama_print_timings:        eval time =   29050.43 ms /    76 runs   (  382.24 ms per token,     2.62 tokens per second)
llama_print_timings:       total time =   33067.44 ms /    91 tokens


[" That's a unique question! While it might be possible to create a partially edible bicycle using ingredients like sugar, honey, or even chocolate (with some creative molding), building a fully functional and safe edible bicycle would likely require significant advances in food engineering and manufacturing. For now, let's just stick to enjoying our favorite treats as snacks rather than trying to build entire vehicles out of them!"]
